대화 데이터셋 링크: https://github.com/beefed-up-geek/HCLT-KACL-2025/blob/main/Korean_Dialouge_Inference/dataset/original/dev_conversations.docx

## 1. 환경 설정

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI         
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
import json
import os

load_dotenv() #만약 .env파일이 없다면, 아래의 코드로 수동으로 API 키 설정
##os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    timeout=60,
    api_key=os.getenv("OPENAI_API_KEY"),        
)

## 2. 한번에 대화기록 통째로 생략된 내용 살리기

### 2.1 시스템 프롬프트 설정

In [18]:
system_prompt_1 = """
너는 대화내용을 분석하는 전문가야. 다음으로 주어지는 대화 내용을 이해하고, 전체 발화에서의 생략된 문장성분을 복구해줘.

예제1.
화자1: 집안일해요?
화자2: 그럼요
화자1: 어떤거요?
화자2: 꾸준히는 아니지만 설거지나 세탁, 청소는 해요
화자2: name1 어떤일을 하시나요?


대답:
화자1: [name 1님은] 집안일해요?
화자2: 그럼요
화자1: [name 1님은] [집안일] 어떤거 [하세]요?
화자2: [저는] 꾸준히는 아니지만 설거지나 세탁, 청소는 해요
화자2: name1 어떤 [집안]일을 하시나요?


예제2.
화자2: 키우고싶은 동물있으세요?
화자2: 아 저는 작은강아지들요 ㅋㅋ종은 크게 상관이없어요.
화자1: 오 저는 아마 안키울것같은데
화자1: 큰개? 키우는 로망?? 만 있어요
화자2: 아아 ㅋㅋㅋㅋ 로망 좋죠ㅋㅋ 큰개도 멋있죠ㅋㅋ

대답: 
화자2: [name1님은] 키우고싶은 동물있으세요?
화자2: 아 저는 작은강아지들요 [키우고 싶어요]ㅋㅋ [강아지의] 종은 크게 상관이없어요.
화자1: 오 저는 아마 [강아지는 실제로] 안키울것같은데
화자1: 큰개? 키우는 로망?? 만 있어요
화자2: 아아 ㅋㅋㅋㅋ [큰개 키우는] 로망 좋죠ㅋㅋ 큰개도 멋있죠ㅋㅋ


"""

integrated_prompt_1 = ChatPromptTemplate.from_messages([
    ("system", system_prompt_1.strip()),
    ("human", '''
     다음은 실제 문제야.
     
     {dialouge}
     
     대답: 
''')
])
chain_1 = integrated_prompt_1 | llm   

### 2.2 실제 담화로 테스트

#### 2.2.1 예제 담화 1

In [ ]:
example_1_1 = '''
화자1: 안녕하세요, 어떤 공연을 주로 관람하시나요?
화자2: 공연은 보고싶은데 기회가 많이 없어서..
화자2: 음 공연 자주 보시나요?
화자1: 코로나 전에는 자주 봤는데 이후로는 없네요ㅠㅠ
화자2: 좋아하는 공연은 뭐였나요
화자1: 저는 주로 콘서트 많이 갔어요, 티켓팅해서요ㅋㅋ
화자2: 티케팅 어렵지않나요?
화자2: 매진속도가.. 와우..
화자1: 사이트 마다 방법이 조금씩 차이가 나서 실패해가며 배웠어요...ㅎㅎ
화자2: 어떤 사이트가 가장 빡세던가요
화자1: 개인적으로 옥션이요...사이트가 너무 연약해요ㅠ
화자2: 아.. 유리서버
화자2: 그럴 때 진짜 화나는데ㅠ
화자2: 어떤 가수의 콘서트를 가보셨나요
화자1: 가수의 콘서트라기보다는 연말 시상식을 주로 갔어요
화자2: 아~연말이 재밌죠
화자2: 볼거리도 많고
화자1: 가수들도 많이 나오니까요

'''
gpt_answer_1_1 = chain_1.invoke({"dialouge":example_1_1}).content.strip()
print(gpt_answer_1_1)

화자2: [name1님은] 음, 공연 자주 보시나요?
화자1: 코로나 전에는 [저는] 자주 봤는데 이후로는 [공연을] 없네요ㅠㅠ
화자2: [name1님이] 좋아하는 공연은 뭐였나요?
화자1: 저는 주로 콘서트 많이 갔어요, [티켓을]팅해서요ㅋㅋ
화자2: [티켓]팅 어렵지 않나요?
화자2: 매진 속도가.. 와우..
화자1: [저는] 사이트마다 방법이 조금씩 차이가 나서 실패해가며 배웠어요...ㅎㅎ


#### 2.2.2 예제 담화 2

In [27]:
example_1_2 = '''
화자1: 안녕하세요 :) 좋아하는 운동종목이 있나요?
화자2: 저는 최근에 수영 배워봤어요~! 수영 좋아합니다 ㅎㅎ
화자2: 수영 좋아하시나요~??
화자1: 와!! 저 수영 엄청 좋아해요!! 새벽반으로 매일 다녔는데ㅠㅠ 지금은 못 가고 있어요ㅠㅠ
화자2: 헐 새벽반 다니셨구나 진짜 대단해요 부지런하시다..
화자2: 수영 다니신지 오래되신건가요?? 저는 막 시작해서 2달다니고 못가고 있어요ㅠ
화자1: 저는 4년정도 다닌 것 같아요! 원래 이렇게 길게 다닐 생각은 없었는데 수영을 다니다 안 다니니까 살이 엄청 쪄서... ㅎㅎ 다른 운동으로는 수영밥 살이 안 빠지더라고요ㅠㅠ 두 달이면 영법 익히고 한참 재밌을 땐데 아쉽네요ㅠㅠ
화자2: 와... 4년 대단하시다!! 수영이 진짜 체력소모가 큰 운동이죠ㅠ
화자2: 맞아요!! 이제 갓 킥판떼고 자유형 하고 슬 배영도 익히는 단계였는데ㅠㅠ
화자2: 코로나가 심해져서 흑흑 ㅠ
화자1: 나중에 코로나 잠잠해지고 스포츠 경기 직관 가능해지면 수영대회 같은거 직관하러 가보세요! 확실히 선수들이라 물 가르는 소리가 다른데 듣고만 있어서 속 시원하고 너무 좋아요!
화자2: 우와 수영대회 직관이런건 생각도 안해봤는데 진짜 한번 가보고싶네요!!!
화자2: 진짜 속이 뻥 뚫릴거 같아요!!
화자2: 이번 올림픽에서도 수영하는거 봤는데 티비로 보는것만으로도 좋았는데
화자2: 직관하면 얼마나 좋을까 정말 궁금하네요!
화자1: 정말정말 추천합니다! 아니 사람이 저런 소리를 내는데 고래들은 얼마나 시원시원하게 물살을 가를까싶어요 막 ㅋㅋㅋㅋ 가끔 성인들이랑 애기들 대회를 같이 할 때가 있는데 애기들은 진짜 너무... 너무 귀엽습니다..
화자2: 악 세상에!!!! 정말 보러가야겠어요!! 엌ㅋㅋㅋㅋㅋㅋ 애기들 애기들 진짜 잘하던데 쪼꼬만게ㅠㅠ
화자2: 수영 경기 티켓 끊는건 쉽나요??! 저 진짜 코로나 없어지면 보러가고싶은데
화자1: 수영은 꽤 쉬운 편인 것 같아요! 일반부 수영대회는 무료입장인 곳도 많고, 선수들 대회도 저는 티켓팅보다 그냥 입구에서 티켓 끊고 갔었던 것 같아요 ㅎㅎ
화자2: 오오 그렇구나! 다행이네요! 새로운 정보 알려주셔서 감사해요ㅠㅠ
화자2: 수영경기 말고 다른경기도 보신적 있으세요?? 저는 최근에 배구경기 보러가고싶더라구요!!
화자1: 세상에! 저도 배구직관 할 날만을 손꼽아 기다리고 있어요ㅠㅠ 도쿄올림픽 이후로 여자배구에 푹 빠져있답니다~ 배구는 피켓팅이라던데 조금 걱정되네요ㅠㅠ
'''
gpt_answer_1_2 = chain_1.invoke({"dialouge":example_1_2}).content.strip()
print(gpt_answer_1_2)

화자1: 안녕하세요 :) [name2님은] 좋아하는 운동종목이 있나요?
화자2: 저는 최근에 수영 [을] 배워봤어요~! 수영 [을] 좋아합니다 ㅎㅎ
화자2: [name1님은] 수영 좋아하시나요~??
화자1: 와!! 저 수영 엄청 좋아해요!! 새벽반으로 매일 다녔는데ㅠㅠ 지금은 [수영을] 못 가고 있어요ㅠㅠ
화자2: 헐 [name1님은] 새벽반 다니셨구나 진짜 대단해요 [정말] 부지런하시다..
화자2: [name1님은] 수영 다니신지 오래되신건가요?? 저는 막 시작해서 2달 [동안] 다니고 못가고 있어요ㅠ
화자1: 저는 4년정도 다닌 것 같아요! 원래 이렇게 길게 다닐 생각은 없었는데 수영을 다니다 [가] 안 다니니까 살이 엄청 쪄서... ㅎㅎ 다른 운동으로는 수영 [을] 하니 살이 안 빠지더라고요ㅠㅠ 두 달이면 영법 익히고 한참 재밌을 땐데 아쉽네요ㅠㅠ
화자2: 와... 4년 [다니신 거] 대단하시다!! 수영이 진짜 체력소모가 큰 운동이죠ㅠ
화자2: 맞아요!! 이제 갓 킥판 [을] 떼고 자유형 [을] 하고 슬 [로] 배영도 익히는 단계였는데ㅠㅠ
화자2: 코로나가 심해져서 흑흑 ㅠ
화자1: 나중에 코로나 잠잠해지고 스포츠 경기 [를] 직관 가능해지면 수영대회 같은거 [를] 직관하러 가보세요! 확실히 선수들이라 물 가르는 소리가 다른데 듣고만 있어서 속 시원하고 너무 좋아요!
화자2: 우와 수영대회 직관 [은] 이런건 생각도 안해봤는데 진짜 한번 가보고싶네요!!!
화자2: 진짜 속이 뻥 뚫릴거 같아요!!
화자2: 이번 올림픽에서도 수영하는거 봤는데 티비로 보는것만으로도 좋았는데
화자2: 직관하면 얼마나 좋을까 정말 궁금하네요!
화자1: 정말정말 추천합니다! 아니 사람이 저런 소리를 내는데 고래들은 얼마나 시원시원하게 물살을 가를까 싶어요 막 ㅋㅋㅋㅋ 가끔 성인들이랑 애기들 대회를 같이 할 때가 있는데 애기들은 진짜 너무... 너무 귀엽습니다..
화자2: 악 세상에!!!! 정말 보러가야겠어요!! 엌ㅋㅋㅋㅋㅋㅋ 애기들 애기들 진짜 잘하던데 쪼꼬만게ㅠㅠ
화자2: 수영 

#### 2.2.3 예제 담화 3

In [28]:
example_1_3 = '''
화자1: 최근에 구매하고 싶으신 물건 있으셨나요?
화자2: 네 전 핸드폰이요..!
화자2: 고민하다가 이번 갤럭시 폴더블 사전예약으로 결국 구매했다네요..ㅎㅎ
화자1: 헐 너무 멋져요
화자2: 이후론 저희 고양이 간식 정도이려나요..
화자1: 저도 갤럭시워치 사고 싶어서 엄청 고민하고 있거든요
화자1: 이번에 삼성에서 나온 것들이 너무 예쁜 것 같아요
화자2: 앗 저도 갤워치 너무 사고 싶었는데
화자2: 맞아요 다들 디자인 너무 이쁘게 나와가지고ㅜ
화자1: 이번에 기능들도 대박이더라구요
화자1: 유튜브에서 워치로 결제하는 거 보고
화자1: 정말 확 끌려서 너무 사고싶었어요 ㅠㅠ
화자2: 와 그거 진짜 엄청 편할것 같아요
화자2: 저도 그동안은 스마트워치쪽은 그다지 관심없었는데
화자2: 이번에 나온거 보고는 뽐뿌차서ㅜㅜ
화자2: 사고싶지만 이미 폰도 바꿔서 뇌에 힘 주고 있어요..
화자1: 맞아요 사실 기능이나 디자인 같은건 애플워치 쪽이 취향인데
화자1: 폰이 갤럭시라 쓸 수가 없네요...
화자2: 앗.. 그건 조금 슬픈 일이네요..ㅜ
화자1: ㅠㅠ 그쵸
화자1: 저는 두 브랜드 다 좋아하는 편이라
화자1: 폰은 삼성 태블릿은 애플 이런식으로 다양하게 쓰고 있어요 ㅎㅎ
화자2: 앗 저두 폰은 삼성 태블릿은 아이패드 이렇게 쓰고 있어요 ㅋㅋㅋ
화자1: 우와 ㅋㅋㅋㅋ 완전 우연이네요
화자1: 생각보다 이렇게 쓰시는 분 많은 것 같아요!
화자1: 사실 전 갤탭 쓰다가 갈아탔거든요ㅎㅎ..
화자2: 마자요 이런 식으로 많이들 쓰시더라구요 ㅋㅋㅋ
화자2: 헐 저도 아이패드 쓰기 전에 갤탭 썼었어요...
화자2: 대박ㅋㅋㅋㅋ
'''
gpt_answer_1_3 = chain_1.invoke({"dialouge":example_1_3}).content.strip()
print(gpt_answer_1_3)

화자1: [name2님은] 최근에 구매하고 싶으신 물건 있으셨나요?
화자2: 네 [저는] 핸드폰이요..!
화자2: 고민하다가 이번 갤럭시 폴더블 사전예약으로 결국 구매했다네요..ㅎㅎ
화자1: 헐 [그거] 너무 멋져요
화자2: 이후론 [저의] 고양이 간식 정도이려나요..
화자1: 저도 갤럭시워치 사고 싶어서 엄청 고민하고 있거든요
화자1: 이번에 삼성에서 나온 것들이 너무 예쁜 것 같아요
화자2: 앗 [저도] 갤워치 너무 사고 싶었는데
화자2: 맞아요 [삼성의] 디자인 너무 이쁘게 나와가지고ㅜ
화자1: 이번에 기능들도 대박이더라구요
화자1: 유튜브에서 워치로 결제하는 거 보고
화자1: 정말 확 끌려서 너무 사고싶었어요 ㅠㅠ
화자2: 와 [그 기능은] 진짜 엄청 편할 것 같아요
화자2: 저도 그동안은 스마트워치쪽은 그다지 관심없었는데
화자2: 이번에 나온 거 보고는 뽐뿌 차서ㅜㅜ
화자2: 사고 싶지만 이미 폰도 바꿔서 [지금은] 뇌에 힘 주고 있어요..
화자1: 맞아요 사실 기능이나 디자인 같은 건 애플워치 쪽이 취향인데
화자1: 폰이 갤럭시라 쓸 수가 없네요...
화자2: 앗.. [그것은] 조금 슬픈 일이네요..ㅜ
화자1: ㅠㅠ [그건] 그쵸
화자1: 저는 두 브랜드 다 좋아하는 편이라
화자1: 폰은 삼성 태블릿은 애플 [로] 이런식으로 다양하게 쓰고 있어요 ㅎㅎ
화자2: 앗 [저도] 폰은 삼성 태블릿은 아이패드 이렇게 쓰고 있어요 ㅋㅋㅋ
화자1: 우와 ㅋㅋㅋㅋ [그건] 완전 우연이네요
화자1: 생각보다 이렇게 쓰시는 분 많은 것 같아요!
화자1: 사실 전 갤탭 쓰다가 갈아탔거든요ㅎㅎ..
화자2: 맞아요 이런 식으로 많이들 쓰시더라구요 ㅋㅋㅋ
화자2: 헐 저도 아이패드 쓰기 전에 갤탭 썼었어요...
화자2: 대박ㅋㅋㅋㅋ


### 

## 3. 한번에  한 문장씩 생략된 내용 살리기

### 3.1 시스템 프롬프트 설정

In [ ]:
system_prompt_2 = """
너는 대화내용을 분석하는 전문가야. 다음으로 주어지는 대화 내용을 이해하고, 마지막 발화에서의 생략된 문장성분을 복구해줘. 만약 생략된 문장성분이 없다면, 그대로 출력해줘.

예제1.
화자1: 면접 / 있디요 / 8월에
화자2: 아하아하 / 일단 7월이 핵심이네
화자1: 근데 결과는 9월이라

대답: 근데 결과는 9월이라 [더 기다려야 한다.]

예지2.
화자1: 그런게 젤 ㅣ싫어
화자2: 공무원이라[서 그래.]
화자1: 합격해서 순천에서 살고 싶다.

대답: 합격해서 순천에서 살고 싶다.

예제3.
화자1: 개좋을듯
화자2: 뭔가 / 요즘 들어 / 내 고향이 좋아.

대답: 뭔가 / [나는] 요즘 들어 / 내 고향이 좋아.
"""

integrated_prompt_2 = ChatPromptTemplate.from_messages([
    ("system", system_prompt_2.strip()),
    ("human", '''
     이제 아래의 실제 대화의 마지막 발화에서 생략된 부분을 모두 채워서, 마지막 문장만 반환해줘. 
     {dialouge}
     
     대답: 
''')
])
chain_2 = integrated_prompt_2 | llm   

### 3.2 실제 대화로 테스트

#### 3.2.1 예제 담화 1 

In [20]:
example_2_1 = '''
화자2: 음 [name1님은] 공연 자주 보시나요?
화자1: 코로나 전에는 [공연을] 자주 봤는데 이후로는 [공연을 보러 간적이] 없네요ㅠㅠ
화자2: [name1님은] 좋아하는 공연은 뭐였나요
화자1: 저는 주로 콘서트 많이 갔어요, [콘서트를] 티켓팅해서요ㅋㅋ
화자2: [콘서트를] 티케팅 [하는 것이] 어렵지않나요?
화자2: [콘서트의] 매진속도가.. 와우..
화자1: 사이트 마다 방법이 조금씩 차이가 나서 실패해가며 배웠어요...ㅎㅎ
'''
gpt_answer_2_1 = chain_2.invoke({"dialouge":example_2_1}).content.strip()
print(gpt_answer_2_1)

사이트 마다 방법이 조금씩 차이가 나서 실패해가며 배웠어요...ㅎㅎ [그래서 티켓팅이 어렵기도 하죠.]


#### 3.2.2 예제 담화 2

In [21]:
example_2_2 = '''
화자2: 헐 [수영학원의] 새벽반 다니셨구나 진짜 대단해요 [name1님은] 부지런하시다..
화자2: [name1님은] 수영 다니신지 오래되신건가요?? 저는 [수영을] 막 시작해서 2달다니고 못가고 있어요ㅠ
화자1: 저는 [수영을] 4년정도 다닌 것 같아요! [저는] 원래 이렇게 길게 다닐 생각은 없었는데 수영을 다니다 안 다니니까 살이 엄청 쪄서... ㅎㅎ 다른 운동으로는 수영밥 살이 안 빠지더라고요ㅠㅠ [수영 다니기 시작한지] 두 달이면 영법 익히고 한참 재밌을 땐데 아쉽네요ㅠㅠ
화자2: 와... 4년 대단하시다!!
'''
gpt_answer_2_2 = chain_2.invoke({"dialouge":example_2_2}).content.strip()
print(gpt_answer_2_2)

와... 4년 대단하시다!! [저도 빨리 수영을 다시 시작하고 싶어요.]


#### 3.2.3 예제 담화 3

In [24]:
example_2_3 = '''
화자1: [name1님은] 최근에 구매하고 싶으신 물건 있으셨나요?
화자2: 네 전 핸드폰이 [구매하고 싶었어]요..!
화자2: [그래서 저는]고민하다가 이번 갤럭시 폴더블 사전예약으로 결국 구매했다네요..ㅎㅎ
화자1: 헐 [갤럭시 폴더블 사전예약으로 구매하는 거] 너무 멋져요
화자2: 이후론 저희 고양이 간식 정도이려나요..
'''
gpt_answer_2_3 = chain_2.invoke({"dialouge":example_2_3}).content.strip()
print(gpt_answer_2_3)

이후론 저희 고양이 간식 정도이려나요. [저는] 구매하고 싶어요.


## 4. 국립국어원 데이터 담화 예제들

In [ ]:
'''
화자1: 안녕하세요! ㅎㅎ
화자2: 안녕하세요!
화자2: 동호회활동하세요?
화자1: 저는 차 마시는 거 좋아해서 차 좋아하는 사람들이랑 가끔 다회 참석해요
화자1: name2님은 동호회활동 참여하는 거 있으신가요?
화자2: 오오
화자2: 아니요ㅋㅋㅋ저는 없어요
화자2: 새로운 사람 만나는 걸 즐기지는 않아서 아직 시도를 못 해봤어요 동호회!
화자1: 아하ㅎㅎ  저는 직업상 만나는 사람들이 한정적이어서 동호회 활동을 하니까 다양한 사람을 만나게 되어 좋더라고요
화자1: 저도 굉장히 내향적인 편인데 동호회 안에서 친해진 사람들하고만 만나고 있어요
화자2: 이상하게 동호회는 목적이 다르다는 편견이 좀 있어서 망설여지더라고요!
화자2: 물론 다 그런 게 아닌데!
화자1: 아 무슨 말씀이신지 너무 알 것 같아요
화자1: 불순한 목적으로 동호회 참석하는 사람들 때문에 좀 이미지가 그렇긴 하죠
화자1: 특히 남자..들.. 이런 말 해도 되나요 ㅋㅋㅋ
화자2: 으으 맞아요 맞아
화자2: 안 좋은 얘기도 사실 많이 들어서...
화자1: 제가 참석하는 차 동호회는 20대 여자가 대부분이라 더 편하게 만날 수 있는 것 같아요ㅠ
화자1: 여성 동호회 쪽으로 알아보시면 좀 걱정이 덜하지 않을까요
화자2: 오! 그런 방법이 있겠네요
화자2: 그리고 어떤 분이 그러셨는데
화자2: 약간 돈이 드는 동호회? 는 오히려 그런 사람이 적대요ㅋㅋㅋㅋㅋㅋ
화자1: 돈이 드는 동호회 ㅋㅋㅋㅋㅋ  차는 돈이 좀 들어서 그런 걸까요
화자2: 네! 그럴지도 몰라요
화자2: 동호회는 인터넷에서 검색해서 들어가는 건가요?
화자1: 저는 트위터로 시작해서 마음 맞는 사람들이랑 같이 만나고 있어요
화자2: 오호 트위터!
화자2: 그런 방법도 있네요 정말
화자2: 트위터하면 아무래도 관심사가 비슷한 사람을 만나게 되니까요
화자1: 맞아요 맞아요

'''

In [ ]:
'''
화자1: 안녕하세요, 어떤 공연을 주로 관람하시나요?
화자2: 공연은 보고싶은데 기회가 많이 없어서..
화자2: 음 공연 자주 보시나요?
화자1: 코로나 전에는 자주 봤는데 이후로는 없네요ㅠㅠ
화자2: 좋아하는 공연은 뭐였나요
화자1: 저는 주로 콘서트 많이 갔어요, 티켓팅해서요ㅋㅋ
화자2: 티케팅 어렵지않나요?
화자2: 매진속도가.. 와우..
화자1: 사이트 마다 방법이 조금씩 차이가 나서 실패해가며 배웠어요...ㅎㅎ
화자2: 어떤 사이트가 가장 빡세던가요
화자1: 개인적으로 옥션이요...사이트가 너무 연약해요ㅠ
화자2: 아.. 유리서버
화자2: 그럴 때 진짜 화나는데ㅠ
화자2: 어떤 가수의 콘서트를 가보셨나요
화자1: 가수의 콘서트라기보다는 연말 시상식을 주로 갔어요
화자2: 아~연말이 재밌죠
화자2: 볼거리도 많고
화자1: 가수들도 많이 나오니까요

'''

In [ ]:
'''
화자1: 3기 신도시에 관해서 아시는거 있으세요?
화자2: 헉 이런 주제가
화자1: 정말 당황스럽네요!
화자2: 예 저희집 근처에 개발하더라고요ㅋㅋㅋ
화자1: 우와 대박!
화자1: 신기해요!
화자1: 오 광주 이천 고양?
화자1: 아니 1차 사전청약은 인천 남양주 성남이네요!
화자1: 성남은 매번 신도시 목록에 들어다는거 같아요
화자2: 아? 1차는 아닌가봐요
화자2: 저는 과천근처 살아요ㅋㅋㅋ
화자2: 관심없는거 들통나버리기
화자1: 앗 이런~~~ 저도 방금 검색해봤네요!
화자1: 저희집도 신도시됐으묜 좋겠네요~~~
화자1: 새아파트 살아보고싶어요!
화자1: 인프라도 좋지 않을까요?
화자2: 맞아요ㅜㅜ 특히 그렇게 크게 짓는 아파트 단지는 인프라가 없다가도 생기겠죠
화자2: ㅋㅋㅋㅋㅋ
화자1: 맞아요! 집 주변에도 그런 신도시가 들어오신다니!
화자1: 좋으시겠엉ㅅ!
화자1: 과천 근처이시면 사실 주변에 인프라가 충분할거같긴 하네요!
화자1: 저는 수웡 살아서 사당가는길에 과천 지나가봣어요!
화자2: 수원이시면 저희 집 거의 근처 네요 전 안양 살아요 ㅎㅎㅎ
화자1: 헐 그러네요~~ 그나저나 3기 신도시 사전청약이 4333가구면 꽤 큰 숫자인거같아요!!
화자2: 그러니까요ㅎㅎㅎ 하지만 내집은 없군요...
화자1: 맞아요.. 3기 신도시에 관심 있으신가요..?ㅎ...
화자2: 집만 준다면야... 무엇이든ㅋㅋㅋㄱ
화자2: 그런데 저는 비혼이라 청약이 될지 모르겠네요
화자1: 앟 저두요..

'''

In [ ]:
'''
화자2: 요즘 보시는 티비프로 같은거 있으신가요?
화자1: 놀면뭐하지 봐요!
화자1: 주말드라마도 가끔 보고 편스토랑도 가끔 보네요 ㅎㅎ
화자2: 오 저도 저번주인가 놀면뭐하니 봤거든요 그 신유빈 선수 나온 편!!
화자2: 넘 웃겼어요ㅋㅋㅋㅋ
화자1: 네네 진짜 웃기더라구요!
화자2: 오 편스토랑은 아직 못봤는데 재밌나요?
화자1: 네네 가볍게 볼만해요 류수영 나오는데 진짜 요리를 쉽고 간편하게 알려주더라구요
화자2: 정말요?? 저 요리배워야되는데ㅋㅋ한번 봐야겠네요
화자1: 네네 지난방송 잘라서 유튜브로 올라오니까 유튜브로 보셔도 충분할거예요!
화자1: 다른 방송 보시는거 있으신가요?
화자2: 대탈출도 보고있어요! 이런 추리물?이 재밌더라구요
화자2: 전에 여고추리반도 엄청 재밌게 본 기억이 있네요ㅎㅎ
화자1: 아맞다 저도 둘다 봐요
화자1: 진짜 재밌죠 저도 그런거 좋아해요!
화자2: 그쵸!! 넘 재밌어요ㅋㅋ
화자1: 저번주 에피소드 역대급이었어요 대탈출
화자2: 아 대박이었죠 진짜 레전드..
화자1: 시즌4에서 지난주가 젤 재밌더라구요
화자2: 맞아요 지난주 진짜 너무 재밌었어요!!ㅠㅠ
화자1: 이번주가 넘 기대됩니다 지난시즌은 몰아봐서 바로바로 볼 수 있었는데 이번엔 본방 보니까 기다리는게 힘드네요
화자2: 진짜요 일주일이 왜이렇게 긴건지.. 빨리 일요일이 되었음 좋겠어요
화자1: 그러게요! 요즘 일주일 낙이 대탈출이네요
'''

In [ ]:
'''
화자2: 가사노동ㅠ
화자1: ㅠㅠㅠ 해도해도 끝이 없는 집안일..
화자2: 마자요
화자2: 돈도안주는데
화자2: 힘들기만하고
화자1: 마자요 ㅠ 돈 좀 많이 받으면서 해야하는 노동 ,ㅠㅠ
화자1: 설거지 특히 넘 싫어용..
화자2: 식기세척기사고싶어요
화자1: 저두요~ 넘 편하고좋을것같아요
화자2: 비싼게단점
화자1: 그쵸 ,, 집에 설치할 자리도 없는..
화자1: 누가 대신 해주면좋겠어용 ㅠㅋㅋ
화자2: 누가대신ㅎㅎㅎㅎ
화자2: 저희집도좁아서
화자1: 흡 ㅠㅋㅋ 근데 또 넓은데 살면 그만큼 가사노동도 많아지겠져 ㅠ?
화자2: 그땐로봇청소기
화자1: 오 좋네용 ㅋㅋ
화자2: 저도갖고싶네여
화자1: 저두요 .. 하나 마련할까하다가도 ㅋㅋ 고민하게되네용
'''

In [ ]:
'''
화자1: 안녕하세요 요즘 완전 가을날씨같아요!
화자2: 안녕하세요~ 천고마비의 계절이죠 ㅎㅎ 하늘이 맑은 날은 더 가을같더라구요
화자1: 네 오늘 강변 산책나가려구요ㅎㅎ 비오더니 맑아졌어요
화자2: 아 너무 좋겠네요 기분전환되고~
화자1: 네ㅎㅎ비와서 바람도 시원하고ㅎㅎ 진짜 가을같아요
화자2: 짧은 가을이니만큼 느낄수있을때 만끽해야죠!
화자1: 네 맞아요 정말 이번에도 금방 겨울올거같아요ㅠㅠ
화자2: 봄가을옷이 참 예쁜데 ㅋㅋ 얼마못입으니 참느라 힘들어요ㅠㅠ
화자1: 맞아요ㅋㅋㅋ고르다가도 참고 패딩사야지ㅠㅜ하고 말아요
화자2: 그쵸 ㅋㅋ 이거 두번만 참으면 패딩이다 하면서 허벅지찌르기 ㅠㅠ
화자1: 그리고 배송도ㅋㅋㅋ지금고르면 담주 날씨를 모르겠어서ㅋㅋ
화자2: 으악 맞아요!!ㅋㅋ 아무리 빨리배송돼도 3일은 잡아야 하는데 그새 날씨가 변해버리죠
화자1: 네ㅜㅜ항상 짧아서 아쉽네요..
'''

In [ ]:
'''
화자1: 안녕하세요~ 직장인이신가용?
화자2: 네, name1님도 직장인이신가요?
화자1: 넵 저두 직장인이에요 ㅋㅋ
화자1: 일 하신지는.얼마나 되셨어요?
화자2: 일 한지는 이제 6년되었어요! name1님은요?
화자1: 저는 이제 4년차요!
화자1: 16년부터 하긴 했는데 중간중간 휴직을 오래해서요 ㅋㅋ
화자2: 아, 그렇군요. 출퇴근은 어느 정도 걸리세요? 저는 꽤 걸리는 편이라 차라리 회사에 집을 지었으면 하고 생각하기도 해요.
화자1: 저는 한시간 반 정도요ㅠ
화자1: 인천에 사는데 서울로 출퇴근중이에요
화자2: 와, 진짜 출퇴근이 힘드시겠어요. 자차로 출근하세요?
화자1: 아녀 차가 더 밀려가지구 그냥 지하철 타구 다녀요
화자2: 아 그렇군요. 퇴근길 지하철은 진짜 힘드시겠어요.
화자1: 네 1호선은 진짜 고통스러워요ㅠㅠ
화자1: 심지어 핸두폰도 잘 안 터져요ㅠ
화자2: 아, 1호선 진짜 고통스럽죠. 사람들이 많아서 핸드폰도 잘 안 되나봐요.
화자1: 네 그런 것 같아요 ㅋㅋ
화자1: 저는 오늘두.. 출근하러 나왔네용  ㅜㅜ
화자1: 월요일 너무 힘들죠...
화자2: 저두요. 진짜 출퇴근 길 너무 너무 싫어요. 마법으로 뿅 하고 출퇴근 했으면 좋겠다는 생각마저...
화자1: 진짜요ㅠ 문 열면 회사면 좋겠어요 아니면 적어도 지하철역으로라도...ㅠㅠ
화자2: 맞아요. 아휴, 진짜 너무 사람도 많고 복잡하고... 그나마 다행인 건 비가 안 와서인 것 같아요. 비오면 우산까지 있어서 더 복잡하니...
화자1: 맞아요 미끄럽구 축축하고 춥고...
화자1: 게다가 1호선은 지상철이라 연착돼요ㅠ
화자2: 헐, 정말요? 아 연착까지 된다니 출근할 때는 진짜 조마조마하겠어요.
화자1: 네.. 그래서 그냥 30분정도 늘 일찍 나와요 ㅋㅋ
화자1: 가끔 급행이 사고로 아예 운 행 안 할 때 있는데 그러면 그 날은...ㅋㅋ
화자2: 어휴... 급행이 운행 안 한다니 너무 끔찍하네요. 퇴근길은 이해한다쳐도 출근길은...

'''